# Pathfinding Logic

In [1]:
import folium
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
data_tables = ['calendar_dates', 'calendar', 'routes', 'shapes', 'stop_times', 'stops', 'trips']

In [3]:
# import data preprocessed and saved in feather format

for table in data_tables:
    globals()[table] = pd.read_feather(f'data/{table}.ftr')

In [46]:
# print the shape of a dataframe and value counts operation on each column within

def table_values(df):
    df_shape = df.shape
    df_rows = df_shape[0]
    df_columns = df_shape[1]
    print(f'{df_rows} rows x {df_columns} columns')
    print()
    for column in df:
        print(column.upper())
        print(df[column].value_counts().sort_index(ascending = False).sort_values(ascending = False))
        print()

## Experiment 1 - Defined home stop to defined end stop on same route

Home stop will be the stop closest to my home address: York Mills stop at 1222 York Mills Road. End stop will be closest stop to Longo's grocery store.

In [9]:
home_stop = stops[stops.stop_id == 917]
home_stop

,stop_id,stop_code,stop_name,stop_lat,stop_lon,wheelchair_boarding
452,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,1


In [23]:
home_stop = stops[stops.stop_id == 917]
longos_stop = stops[stops.stop_id == 9974]
my_stops = pd.concat([home_stop, longos_stop]).reset_index()

In [24]:
my_stops

,index,stop_id,stop_code,stop_name,stop_lat,stop_lon,wheelchair_boarding
0,452,917,9083,York Mills Rd at Sandover Dr (1222 York Mills),43.759813,-79.331751,1
1,7990,9974,13469,York Mills Rd at Scarsdale Rd,43.753660,-79.357427,2


### Objective 1 - Find every trip taken (on the stop_times table) which includes both the home stop and the Longo's stop

In [27]:
stop_times

,trip_id,stop_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,42990004,7:00:29,14155,1,0,0,0.1303
1,42990004,7:01:19,3807,2,0,0,0.3546
2,42990004,7:02:12,6904,3,0,0,0.5903
3,42990004,7:03:35,1163,4,0,0,0.9613
4,42990004,7:04:48,7723,5,0,0,1.2849
...,...,...,...,...,...,...,...
4311626,43130109,26:25:00,14535,1,0,0,NaN
4311627,43130109,26:27:05,14536,2,0,0,1.6279
4311628,43130109,26:28:50,14537,3,0,0,2.4244
4311629,43130109,26:30:43,14538,4,0,0,3.2976


In [30]:
stop_times.trip_id.nunique()

# Number of unique trip_id values, each trip_id value is expanded upon with additional data on the trips table

131940

In [34]:
home_trips = stop_times[stop_times.stop_id == 917]
longos_trips = stop_times[stop_times.stop_id == 9974]

In [36]:
home_trips.shape, longos_trips.shape

((695, 7), (820, 7))

In [38]:
mutual_trips = pd.merge(home_trips, longos_trips, on = 'trip_id', how = 'inner')

In [40]:
mutual_trips.shape

(637, 13)

In [41]:
mutual_trips

,trip_id,stop_time_x,stop_id_x,stop_sequence_x,pickup_type_x,drop_off_type_x,shape_dist_traveled_x,stop_time_y,stop_id_y,stop_sequence_y,pickup_type_y,drop_off_type_y,shape_dist_traveled_y
0,43005604,26:52:20,917,53,0,0,17.0625,26:56:38,9974,60,0,0,19.2423
1,43005605,5:16:41,917,53,0,0,17.0625,5:21:38,9974,60,0,0,19.2423
2,43005606,4:46:41,917,53,0,0,17.0625,4:51:38,9974,60,0,0,19.2423
3,43005600,27:52:20,917,53,0,0,17.0625,27:56:38,9974,60,0,0,19.2423
4,43005602,27:22:20,917,53,0,0,17.0625,27:26:38,9974,60,0,0,19.2423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,43114735,24:09:48,917,48,0,0,15.9697,24:14:23,9974,55,0,0,18.1495
633,43114736,23:59:48,917,48,0,0,15.9697,24:04:23,9974,55,0,0,18.1495
634,43114737,23:49:48,917,48,0,0,15.9697,23:54:23,9974,55,0,0,18.1495
635,43114738,23:39:48,917,48,0,0,15.9697,23:44:23,9974,55,0,0,18.1495


In [43]:
mutual_trips_detail = pd.merge(mutual_trips, trips, on = 'trip_id', how = 'inner')

In [47]:
table_values(mutual_trips_detail)

637 rows x 20 columns

TRIP_ID
43005600    1
43086477    1
43086470    1
43086471    1
43086472    1
           ..
43059140    1
43059141    1
43059142    1
43059143    1
43114757    1
Name: trip_id, Length: 637, dtype: int64

STOP_TIME_X
28:22:20    4
27:52:20    4
27:22:20    4
26:52:20    4
14:21:42    3
           ..
20:50:45    1
20:49:45    1
20:43:45    1
20:41:45    1
9:53:42     1
Name: stop_time_x, Length: 454, dtype: int64

STOP_ID_X
917    637
Name: stop_id_x, dtype: int64

STOP_SEQUENCE_X
48    360
39    169
17     69
53     38
38      1
Name: stop_sequence_x, dtype: int64

PICKUP_TYPE_X
0    637
Name: pickup_type_x, dtype: int64

DROP_OFF_TYPE_X
0    637
Name: drop_off_type_x, dtype: int64

SHAPE_DIST_TRAVELED_X
15.9697    360
12.6142    169
4.8020      69
17.0625     38
13.2318      1
Name: shape_dist_traveled_x, dtype: int64

STOP_TIME_Y
26:56:38    4
28:26:38    4
27:56:38    4
27:26:38    4
26:26:38    3
           ..
17:50:58    1
17:46:11    1
17:44:58    1
17:39:58

In [50]:
shape1 = mutual_trips_detail[mutual_trips_detail.shape_id == 888274]
shape2 = mutual_trips_detail[mutual_trips_detail.shape_id == 888281]
shape3 = mutual_trips_detail[mutual_trips_detail.shape_id == 888280]
shape4 = mutual_trips_detail[mutual_trips_detail.shape_id == 888271]
shape5 = mutual_trips_detail[mutual_trips_detail.shape_id == 888276]

In [52]:
table_values(shape1)

360 rows x 20 columns

TRIP_ID
43027101    1
43086508    1
43086506    1
43086505    1
43086504    1
           ..
43059162    1
43059155    1
43059145    1
43059144    1
43114757    1
Name: trip_id, Length: 360, dtype: int64

STOP_TIME_X
10:21:42    3
16:59:42    3
18:44:42    3
15:29:42    3
11:21:42    3
           ..
21:43:45    1
13:32:42    1
21:53:45    1
21:58:45    1
9:53:42     1
Name: stop_time_x, Length: 270, dtype: int64

STOP_ID_X
917    360
Name: stop_id_x, dtype: int64

STOP_SEQUENCE_X
48    360
Name: stop_sequence_x, dtype: int64

PICKUP_TYPE_X
0    360
Name: pickup_type_x, dtype: int64

DROP_OFF_TYPE_X
0    360
Name: drop_off_type_x, dtype: int64

SHAPE_DIST_TRAVELED_X
15.9697    360
Name: shape_dist_traveled_x, dtype: int64

STOP_TIME_Y
15:36:11    3
10:28:11    3
18:51:11    3
8:08:11     3
17:51:11    2
           ..
19:04:11    1
19:07:58    1
19:17:11    1
19:19:23    1
9:58:58     1
Name: stop_time_y, Length: 275, dtype: int64

STOP_ID_Y
9974    360
Name: stop_i

In [54]:
trips[trips.block_id == 1861653]

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
70550,65004,2,43059043,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888273,1
70562,65004,2,43059002,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70578,65004,2,43058994,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70596,65004,2,43058980,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70614,65004,2,43059022,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70634,65004,2,43059096,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70654,65004,2,43059086,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70672,65004,2,43059062,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70686,65004,2,43059157,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1
70696,65004,2,43059129,EAST - 95A YORK MILLS towards PORT UNION,0,1861653,888272,1


In [55]:
trips

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
0,64815,1,42990014,EAST - 10 VAN HORNE towards VICTORIA PARK,0,1858140,886388,1
1,64815,1,42990036,EAST - 10 VAN HORNE towards VICTORIA PARK,0,1858138,886388,1
2,64815,1,42990015,EAST - 10 VAN HORNE towards VICTORIA PARK,0,1858140,886387,1
3,64815,1,42990038,EAST - 10 VAN HORNE towards VICTORIA PARK,0,1858138,886387,1
4,64815,1,42990020,EAST - 10 VAN HORNE towards VICTORIA PARK,0,1858140,886387,1
...,...,...,...,...,...,...,...,...
131935,65027,4,43129887,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,1864322,890357,2
131936,65027,4,43129886,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,1864325,890357,2
131937,65027,4,43129885,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,1864324,890357,2
131938,65027,4,43129884,LINE 4 (SHEPPARD) towards SHEPPARD-YONGE STATION,1,1864323,890357,2


In [56]:
trips.trip_id

0         42990014
1         42990036
2         42990015
3         42990038
4         42990020
            ...   
131935    43129887
131936    43129886
131937    43129885
131938    43129884
131939    43130109
Name: trip_id, Length: 131940, dtype: int64

In [57]:
stop_times

,trip_id,stop_time,stop_id,stop_sequence,pickup_type,drop_off_type,shape_dist_traveled
0,42990004,7:00:29,14155,1,0,0,0.1303
1,42990004,7:01:19,3807,2,0,0,0.3546
2,42990004,7:02:12,6904,3,0,0,0.5903
3,42990004,7:03:35,1163,4,0,0,0.9613
4,42990004,7:04:48,7723,5,0,0,1.2849
...,...,...,...,...,...,...,...
4311626,43130109,26:25:00,14535,1,0,0,NaN
4311627,43130109,26:27:05,14536,2,0,0,1.6279
4311628,43130109,26:28:50,14537,3,0,0,2.4244
4311629,43130109,26:30:43,14538,4,0,0,3.2976


In [59]:
stop_times.trip_id.value_counts()

43095628    120
43001334    120
43001333    120
43038301    120
43001350    120
           ... 
43026847      1
43026844      1
43026833      1
43026841      1
43026845      1
Name: trip_id, Length: 131940, dtype: int64

In [65]:
mutual_trips

,trip_id,stop_time_x,stop_id_x,stop_sequence_x,pickup_type_x,drop_off_type_x,shape_dist_traveled_x,stop_time_y,stop_id_y,stop_sequence_y,pickup_type_y,drop_off_type_y,shape_dist_traveled_y
0,43005604,26:52:20,917,53,0,0,17.0625,26:56:38,9974,60,0,0,19.2423
1,43005605,5:16:41,917,53,0,0,17.0625,5:21:38,9974,60,0,0,19.2423
2,43005606,4:46:41,917,53,0,0,17.0625,4:51:38,9974,60,0,0,19.2423
3,43005600,27:52:20,917,53,0,0,17.0625,27:56:38,9974,60,0,0,19.2423
4,43005602,27:22:20,917,53,0,0,17.0625,27:26:38,9974,60,0,0,19.2423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,43114735,24:09:48,917,48,0,0,15.9697,24:14:23,9974,55,0,0,18.1495
633,43114736,23:59:48,917,48,0,0,15.9697,24:04:23,9974,55,0,0,18.1495
634,43114737,23:49:48,917,48,0,0,15.9697,23:54:23,9974,55,0,0,18.1495
635,43114738,23:39:48,917,48,0,0,15.9697,23:44:23,9974,55,0,0,18.1495


In [66]:
mutual_trips

,trip_id,stop_time_x,stop_id_x,stop_sequence_x,pickup_type_x,drop_off_type_x,shape_dist_traveled_x,stop_time_y,stop_id_y,stop_sequence_y,pickup_type_y,drop_off_type_y,shape_dist_traveled_y
0,43005604,26:52:20,917,53,0,0,17.0625,26:56:38,9974,60,0,0,19.2423
1,43005605,5:16:41,917,53,0,0,17.0625,5:21:38,9974,60,0,0,19.2423
2,43005606,4:46:41,917,53,0,0,17.0625,4:51:38,9974,60,0,0,19.2423
3,43005600,27:52:20,917,53,0,0,17.0625,27:56:38,9974,60,0,0,19.2423
4,43005602,27:22:20,917,53,0,0,17.0625,27:26:38,9974,60,0,0,19.2423
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,43114735,24:09:48,917,48,0,0,15.9697,24:14:23,9974,55,0,0,18.1495
633,43114736,23:59:48,917,48,0,0,15.9697,24:04:23,9974,55,0,0,18.1495
634,43114737,23:49:48,917,48,0,0,15.9697,23:54:23,9974,55,0,0,18.1495
635,43114738,23:39:48,917,48,0,0,15.9697,23:44:23,9974,55,0,0,18.1495


In [69]:
home_to_longos = trips[trips.trip_id.isin(mutual_trips.trip_id)]

In [70]:
home_to_longos

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,bikes_allowed
15477,64914,1,43005606,WEST - 395 YORK MILLS towards YORK MILLS STATION,1,1860387,887271,1
15478,64914,1,43005605,WEST - 395 YORK MILLS towards YORK MILLS STATION,1,1860353,887271,1
15479,64914,1,43005604,WEST - 395 YORK MILLS towards YORK MILLS STATION,1,1860196,887271,1
15480,64914,1,43005602,WEST - 395 YORK MILLS towards YORK MILLS STATION,1,1860187,887271,1
15481,64914,1,43005600,WEST - 395 YORK MILLS towards YORK MILLS STATION,1,1860136,887271,1
...,...,...,...,...,...,...,...,...
129190,65004,4,43114729,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863747,888274,1
129191,65004,4,43114728,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863787,888274,1
129192,65004,4,43114727,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863746,888274,1
129193,65004,4,43114726,WEST - 95 YORK MILLS towards YORK MILLS STATION,1,1863719,888274,1


In [73]:
table_values(home_to_longos[home_to_longos.route_id == 64914])

38 rows x 8 columns

ROUTE_ID
64914    38
Name: route_id, dtype: int64

SERVICE_ID
4    12
3    11
2     9
1     6
Name: service_id, dtype: int64

TRIP_ID
43005600    1
43098752    1
43071284    1
43071285    1
43071288    1
43071289    1
43071291    1
43098750    1
43098751    1
43098753    1
43005602    1
43098754    1
43098755    1
43098756    1
43098757    1
43098758    1
43098759    1
43098760    1
43071283    1
43071282    1
43071281    1
43071280    1
43005603    1
43005604    1
43005605    1
43005606    1
43041808    1
43041809    1
43041810    1
43041812    1
43041813    1
43041814    1
43041815    1
43041816    1
43041817    1
43071276    1
43071279    1
43098761    1
Name: trip_id, dtype: int64

TRIP_HEADSIGN
WEST - 395 YORK MILLS towards YORK MILLS STATION    38
Name: trip_headsign, dtype: int64

DIRECTION_ID
1    38
Name: direction_id, dtype: int64

BLOCK_ID
1863591    3
1862749    2
1861668    2
1863753    2
1863752    2
1863743    2
1862763    2
1862756    2
1862755    2

In [71]:
table_values(home_to_longos)

637 rows x 8 columns

ROUTE_ID
65004    599
64914     38
Name: route_id, dtype: int64

SERVICE_ID
1      196
2      169
4      136
3      135
401      1
Name: service_id, dtype: int64

TRIP_ID
43005600    1
43086477    1
43086470    1
43086471    1
43086472    1
           ..
43059140    1
43059141    1
43059142    1
43059143    1
43114757    1
Name: trip_id, Length: 637, dtype: int64

TRIP_HEADSIGN
WEST - 95 YORK MILLS towards YORK MILLS STATION     530
WEST - 95C YORK MILLS towards YORK MILLS STATION     69
WEST - 395 YORK MILLS towards YORK MILLS STATION     38
Name: trip_headsign, dtype: int64

DIRECTION_ID
1    637
Name: direction_id, dtype: int64

BLOCK_ID
1860192    14
1860188    12
1860136    12
1861640    12
1863720    11
           ..
1860353     1
1860176     1
1860177     1
1860180     1
1859481     1
Name: block_id, Length: 97, dtype: int64

SHAPE_ID
888274    360
888281    169
888280     69
887271     38
888276      1
Name: shape_id, dtype: int64

BIKES_ALLOWED
1    637
N